# SCREENER DE ACTIVOS CASTIGADOS

Permite determinar activos de renta variable o sectores (vía ETF) que se encuentren más castigados o bajos de precio respecto de sus máximos en una ventana de tiempo n parametrizable. Facilita, así, generar el "margen de seguridad".

## INDICE DE FUNCIONES:
 - getCastigados: Toma argumento "lideres" (acciones líderes ARG), "general" (acciones del panel general ARG), "cedears" (todos los cedears de ARG), "adrs"(los adrs argentinos) o "sectores" (los etfs de los distintos sectores); y una cantidad de n días hacia atrás como ventana para tomar el máximo histórico, y devuelve un listado de activos ordenados del más al menos castigado con los % de caída desde el último ATH de la ventana.

In [2]:
# FUNCIONES NECESARIAS PARA OBTENER DATA FINANCIERA:

def getDataYf(ticker, tipo, interval, data_from = None, data_to = None, period = None):
    """
    Es una función para descargar market data de Yahoo Finance con la librería yfinance.
    
    ## Inputs:
        >ticker: el nombre del ticker.
        >tipo: si es "no end" no se indica hasta cuándo (data_to), se obtiene hasta el último día disponible. Si es "end" es
        necesario indicar hasta cuánto (data_to). En ambos casos hay que indicar desde qué fecha (data_from). Si es "period" 
        no se indica ni desde cuándo ni hasta cuándo, sólo el argumento "period" con la cantidad de tiempo a obtener.
        >now : si es True, no se indica hasta cuándo (data_to), se obtiene hasta el último día disponible. Si se indica False, es
        necesario indicar hasta cuánto (data_to).
        >interval: el timeframe (ej. 1mo, 1h, 1d, 1wk, etc)
        >data_from: data desde qué fecha.
        >data_to: data hasta qué fecha (no inclusive el día). Sólo es aplicable si now == True.
        >period : en caso de tipo = "period", se pasa este argumento que refiere a la cantidad de tiempo a obtener. Ej. 1y, 2y, 3y, etc.
        
    ## Outputs:
        >series OHLC ajustadas del ticker.
    """
    import yfinance as yf
    import pandas as pd
    
    if tipo == "no end":
        data = yf.download(ticker, start = data_from, interval = interval, progress = False, auto_adjust = True)
    elif tipo == "end":
        data = yf.download(ticker, start = data_from, end = data_to, interval = interval, progress = False, auto_adjust = True)
    elif tipo == "period":
        data = yf.download(ticker, interval = interval, period = period, progress = False, auto_adjust = True)
    return data



def getDataYfMulti(activos, tipo, interval, data_from = None, data_to = None, period = None, swap = True):
    """
    Función para hacer batch requests (varios tickers a la vez), que será la fx que más voy a utilizar para market data.
    
    ## Inputs:
        >tickers: es una lista con los tickers de los cuales se va a obtener market data.
        >tipo: si es "no end" no se indica hasta cuándo (data_to), se obtiene hasta el último día disponible. Si es "end" es
        necesario indicar hasta cuánto (data_to). En ambos casos hay que indicar desde qué fecha (data_from). Si es "period" 
        no se indica ni desde cuándo ni hasta cuándo, sólo el argumento "period" con la cantidad de tiempo a obtener.
        >now : si es True, no se indica hasta cuándo (data_to), se obtiene hasta el último día disponible. Si se indica False, es
        necesario indicar hasta cuánto (data_to).
        >interval: el timeframe (ej. 1mo, 1h, 1d, 1wk, etc)
        >data_from: data desde qué fecha.
        >data_to: data hasta qué fecha (no inclusive el día). Sólo es aplicable si now == True.
        >period : en caso de tipo = "period", se pasa este argumento que refiere a la cantidad de tiempo a obtener. Ej. 1y, 2y, 3y, etc.
        >swap : si es True, divide el df en tickers y cada uno tiene su OHLC. Si es False, tenemos cada columna OHLC y dentro todos los tickers.
        
    ## Outputs:
        >series OHLC ajustadas del ticker.
    """
    import yfinance as yf
    import pandas as pd
    
    lideres_arg = ["ALUA.BA", "BBAR.BA", "BMA.BA", "BYMA.BA", "CEPU.BA", "COME.BA", "CRES.BA", "CVH.BA", "EDN.BA", 
                   "GGAL.BA", "LOMA.BA", "MIRG.BA", "PAMP.BA", "SUPV.BA", "TECO2.BA", "TGNO4.BA", "TGSU2.BA", "TRAN.BA", 
                   "TXAR.BA", "VALO.BA", "YPFD.BA"]

    general_arg = ["AGRO.BA", "AUSO.BA", "BHIP.BA", "BOLT.BA", "BPAT.BA", "CADO.BA", "CAPX.BA", "CARC.BA", "CECO2.BA", 
                   "CELU.BA", "CGPA2.BA", "CTIO.BA", "DGCU2.BA", "FERR.BA", "FIPL.BA", "GAMI.BA", "GCDI.BA", "GCLA.BA", 
                   "HARG.BA", "HAVA.BA", "INVJ.BA", "IRSA.BA", "LEDE.BA", "LONG.BA", "METR.BA", "MOLA.BA", "MOLI.BA", 
                   "MORI.BA", "OEST.BA", "PATA.BA", "RICH.BA", "RIGO.BA", "SAMI.BA", "SEMI.BA"]

    cedears = ["AAL", "AAPL", "ABBV", "ABEV", "ABNB", "ABT", "ADBE", "ADGO", "ADI", "ADP", "AEM", "AIG", "AMAT", "AMD", 
               "AMGN", "AMZN", "AOCA", "ARCO", "ARKK", "ASR", "AUY", "AVGO", "AXP", "AZN", "BA", "BA.C", "BABA", "BB", 
               "BBD", "BBV", "BCS", "BHP", "BIDU", "BIIB", "BIOX", "BITF", "BK", "BMY", "BNG", "BP", "BRFS", "BRKB", "BSBR", 
               "C", "CAAP", "CAH", "CAR", "CAT", "CBRD", "CDE", "CL", "COIN", "COST", "CRM", "CS", "CSCO", "CVX", "CX", "DD", 
               "DE", "DESP", "DIA", "DISN", "DOCU", "DOW", "E", "EA", "EBAY", "EEM", "EFX", "ERIC", "ERJ", "ETSY", "EWZ", "F", 
               "FCX", "FDX", "FMX", "FSLR", "GE", "GFI", "GGB", "GILD", "GLOB", "GLW", "GM", "GOLD", "GOOGL", "GPRK", "GRMN", 
               "GS", "HAL", "HD", "HL", "HMC", "HMY", "HOG", "HON", "HPQ", "HSBC", "HSY", "HUT", "HWM", "IBM", "IFF", "INTC", 
               "ITUB", "IWM", "JD", "JMIA", "JNJ", "JPM", "KMB", "KO", "KOFM", "LLY", "LMT", "LRCX", "LVS", "LYG", "MA", "MCD", 
               "MDT", "MELI", "META", "MMM", "MO", "MOS", "MRK", "MSFT", "MSI", "MSTR", "MU", "NEM", "NFLX", "NGG", "NIO", "NKE", 
               "NOKA", "NTCO", "NTES", "NUE", "NVDA", "NVS", "ORAN", "ORCL", "OXY", "PAAS", "PAC", "PANW", "PBI", "PBR", "PCAR", 
               "PEP", "PFE", "PG", "PHG", "PKS", "PSX", "PYPL", "QCOM", "QQQ", "RBLX", "RIO", "RTX", "SAN", "SAP", "SATL", "SBUX", 
               "SCCO", "SE", "SHEL", "SHOP", "SI", "SID", "SLB", "SNAP", "SNOW", "SONY", "SPGI", "SPOT", "SPY", "SQ", "SYY", "T", 
               "TEFO", "TEN", "TGT", "TM", "TMO", "TRIP", "TRVV", "TSLA", "TSM", "TTE", "TV", "TWLO", "TXN", "TXR", "UAL", "UBER", 
               "UGP", "UL", "UNH", "UNP", "UPST", "USB", "V", "VALE", "VIST", "VIV", "VOD", "VZ", "WBA", "WFC", "WMT", "X", "XLE", 
               "XLF", "XOM", "XP", "YY", "ZM"]

    adrs = ["BBAR", "BMA", "CEPU", "CRESY", "EDN", "GGAL", "IRS", "LOMA", "PAM", "SUPV", "TEO", "TGS", "TS", "TX", "YPF"]

    sectors = ["XLC", "XLP", "XLY", "XLF", "XLV", "XLI", "XLRE", "XLU", "XBI", "XLB", "XLK", "XLE"]
    
    precarga = ["lideres", "general", "cedears", "adrs", "sectores"]
    precarga_dict = {"lideres" : lideres_arg, "general" : general_arg, "cedears" : cedears, "adrs" : adrs, "sectores" : sectors}
    
    if activos in precarga:
        activos = precarga_dict[activos]
    
    if tipo == "no end":
        data = yf.download(activos, start = data_from, interval = interval, progress = False, auto_adjust = True)
    elif tipo == "end":
        data = yf.download(activos, start = data_from, end = data_to, interval = interval, progress = False, auto_adjust = True)
    elif tipo == "period":
        data = yf.download(activos, interval = interval, period= period, progress = False, auto_adjust = True)
    
    if swap:
        #data = data.swaplevel(i = 0, j = 1, axis = 1)
        # Algoritmo para procesar el MultipleTicker download de yfinance
        dicto = {}
        low = data["Low"]
        high = data["High"]
        close = data["Close"]
        open = data["Open"]
        volume = data["Volume"]

        tickers = list(data["Close"].columns)

        for ticker in tickers:
            dicto[ticker] = {
                "Open" : open[ticker],
                "High" : high[ticker],
                "Low" : low[ticker],
                "Close" : close[ticker],
                "Volume" : volume[ticker]
            }

            dicto[ticker] = pd.DataFrame(dicto[ticker])
        return dicto
    return data

In [3]:
# FUNCIONALIDADES PRINCIPALES DEL MÓDULO:
# SCREENER ACTIVOS CASTIGADOS: GENÉRICO

def getCastigados(activos, dias, moneda = "pesos"):
    """
    Toma argumento "lideres" (acciones líderes ARG), "general" (acciones del panel general ARG), "cedears" (todos los cedears de ARG), "adrs"
    (los adrs argentinos) o "sectores" (los etfs de los distintos sectores); 
    y una cantidad de n días hacia atrás como ventana para tomar el máximo histórico, y devuelve un listado de activos ordenados del más 
    al menos castigado con los % de caída desde el último ATH de la ventana.
    
    #Inputs:
     - activos: "lideres" (acciones líderes ARG), "general" (acciones del panel general ARG), "cedears" (todos los cedears de ARG), "adrs"
     (los adrs argentinos) o "sectores" (los etfs de los distintos sectores);
     - dias: cantidad de n días hacia atrás como ventana para tomar el máximo histórico.
     - moneda : pesos ("pesos") o dólar CCL ("dólar CCL"). Se puede elegir entre analizar en pesos o en dólares CCL.
     
     #Outputs:
     - devuelve un listado de activos ordenados del más al menos castigado con los % de caída desde el último ATH de la ventana
    """
    
    import yfinance as yf
    import pandas as pd
    import datetime as datetime
    import numpy as np
    from datetime import datetime, timedelta
    import time
    import json
    
    sectors = ["XLC", "XLP", "XLY", "XLF", "XLV", "XLI", "XLRE", "XLU", "XBI", "XLB", "XLK", "XLE"]
    
    desc = {"XLC" : "communications", "XLP" : "consumer staples", 
        "XLY" : "consumer discrecionary", "XLF" : "financial", 
        "XLV": "health care", "XLI" : "industrials", 
        "XLRE": "real estate", "XLU" : "utilities", 
        "XBI" : "biotech", "XLB" : "materials", 
        "XLK": "technology", "XLE" : "energy"}

    desde = (datetime.now() - timedelta(days = dias)).strftime("%Y-%m-%d")

    data = getDataYfMulti(activos, tipo = "no end", data_from = desde, interval = "1d", swap = False)["Close"]
    if moneda == "dolares":
        local = getDataYf("YPFD.BA", tipo = "no end", data_from = desde, interval = "1d")["Close"]
        adr = getDataYf("YPF", tipo = "no end", data_from = desde, interval = "1d")["Close"]
        ccl = (local / adr).to_frame().round(2)
        ccl.columns = ['CCL']
        data = data.div(ccl['CCL'], axis=0)
    
    data.index = pd.to_datetime(data.index).strftime("%Y-%m-%d")
    caidas = {}
    
    for activo in data.columns:
        try:
            maximo = round(data[activo].max(), 2)
            fecha_maximo = data[activo].idxmax()
            fecha_maximo = datetime.strptime(fecha_maximo, "%Y-%m-%d")         # Convertimos string en datetime
            actual = round(data[activo][-1], 2)

            caida = round((((actual/maximo) - 1)*100), 2)
            upside = round((((maximo/actual) - 1)*100), 2)
            minimo = round(data[activo].min(), 2)
            downside = round((((minimo/actual) - 1)*100), 2)
            diasDesdeMax = (datetime.now() - fecha_maximo).days
            # DEBUG: print(f"ACTIVO: {activo}, MÁXIMO: {maximo}, FECHA_MÁXIMO: {fecha_maximo}, ACTUAL: {actual}, CAIDA: {caida}")
            caidas[activo] = {}
            if activos == "sectores":
                caidas[activo]["desc"] = desc[activo]
            caidas[activo]["fechaMax"] = fecha_maximo
            caidas[activo][f"maximo_{dias}dias"] = maximo
            caidas[activo]["diasDesdeUltimoMax"] = diasDesdeMax
            caidas[activo]["precioActual"] = actual
            caidas[activo]["caida"] = caida
            caidas[activo]["upside"] = upside
            caidas[activo]["minimo"] = minimo
            caidas[activo]["downside"] = downside
            # DEBUG: print(activo, caidas[activo])
        except:
            continue
       
    tabla = pd.DataFrame.from_dict(caidas)
    #tabla = pd.DataFrame()
    #tabla.index = caidas.keys()
    #tabla["% CAIDA"] = caidas.values()
    #tabla.sort_values(by="% CAIDA", inplace = True, ascending = True)
    tabla = tabla.transpose().sort_values(by="caida", ascending = True)
    estilados = tabla.style.background_gradient(axis=0, gmap=-tabla['caida'], cmap='Reds') \
    .format('{:.2f}', subset=[f"maximo_{dias}dias", "precioActual", "minimo"])   \
    .format('{:.2f}%', subset=["caida", "upside", "downside"]) \
    .format(lambda t: t.strftime("%d-%m-%Y"), subset=["fechaMax"])
    return estilados
    

#data = getCastigados("lideres", 720):
#data.to_excel("C:\\Users\\user\\Desktop\\ENTORNO DS\\activos_lideres.xlsx", sheet_name='ESCANEO_CAIDAS')  

In [8]:
datos = getCastigados(activos = "sectores", dias = 720)
datos

,desc,fechaMax,maximo_720dias,diasDesdeUltimoMax,precioActual,caida,upside,minimo,downside
XLU,utilities,12-09-2022,74.48,498,60.67,-18.54%,22.76%,55.67,-8.24%
XLRE,real estate,20-04-2022,47.48,643,38.88,-18.11%,22.12%,31.85,-18.08%
XLE,energy,14-09-2023,91.79,131,80.47,-12.33%,14.07%,61.94,-23.03%
XBI,biotech,09-02-2022,97.24,713,89.01,-8.46%,9.25%,62.80,-29.45%
XLY,consumer discrecionary,29-03-2022,188.54,665,174.12,-7.65%,8.28%,125.18,-28.11%
XLP,consumer staples,20-04-2022,76.85,643,71.70,-6.70%,7.18%,64.32,-10.29%
XLB,materials,20-04-2022,87.00,643,82.47,-5.21%,5.49%,65.50,-20.58%
XLF,financial,09-02-2022,39.45,713,38.11,-3.40%,3.52%,29.54,-22.49%
XLI,industrials,28-12-2023,114.13,26,113.32,-0.71%,0.71%,81.01,-28.51%
XLV,health care,10-01-2024,140.97,13,140.07,-0.64%,0.64%,116.48,-16.84%
